# Memory Enhanced AI Agent

This notebook focuses on building an AI agent with persistent memory capabilities and personality.

## What we will build in this notebook:
1. **Agent Personality** - Consistent character and role definition
2. **Memory Learning** - Automatically save new conversations to memory
3. **Enhanced Agent Functions** - Advanced memory management and conversation handling
4. **Interactive Agent** - Natural conversation flow with context continuity


In [4]:
#  we will Import libraries and connect to existing memory system the chroma db

import chromadb
import ollama
from datetime import datetime
import json
import uuid

print(" Building Memory Enhanced AI Agent")
print("=" * 60)

# Connect to our persistent memory from Phase 1 & 2
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_collection(name="conversation_memory")

#above is the same memory collection that we are using

print(" Connected to persistent memory system")
print(f" Current memories stored: {collection.count()}")
print(f" Collection name: {collection.name}")


 Building Memory Enhanced AI Agent
 Connected to persistent memory system
 Current memories stored: 10
 Collection name: conversation_memory


In [15]:
# we are Creating an  AI Agent Class with Personality and Memory Learning

class MemoryEnhancedAgent:
    def __init__(self, name="Bruno", personality="Brain_Powerhouse"):
        self.name = name
        self.personality = personality
        self.model = "llama3.2:3b"
        self.collection = collection  # Use the ChromaDB collection
        
        # Agent personality definitions
        self.personalities = {
            "Brain_Powerhouse": {
                "role": "experienced technical mentor and coding companion",
                "traits": [
                    "Patient and encouraging",
                    "Explains complex concepts clearly", 
                    "Remembers your learning journey",
                    "Builds on your existing knowledge",
                    "Provides practical examples"
                ],
                "style": "friendly but professional, uses examples from your tech interests"
            },
            "creative_problem_solver": {
                "role": "innovative problem-solving partner",
                "traits": [
                    "Thinks outside the box",
                    "Connects ideas creatively",
                    "Remembers your project goals",
                    "Suggests novel approaches"
                ],
                "style": "enthusiastic and creative, makes unexpected connections"
            }
        }
        
        print(f" AI Agent '{self.name}' initialized!")
        print(f" Personality: {personality}")
        print(f" Connected to memory with {self.collection.count()} stored conversations")
    
    def get_personality_prompt(self):
        """Get the personality-specific system prompt"""
        personality_info = self.personalities.get(self.personality, self.personalities["Brain_Powerhouse"])
        
        prompt = f"""You are {self.name}, a {personality_info['role']} with persistent memory.

PERSONALITY TRAITS:
{chr(10).join(f"- {trait}" for trait in personality_info['traits'])}

COMMUNICATION STYLE: {personality_info['style']}

CORE BEHAVIOR:
- Always maintain this personality consistently across conversations
- Reference past conversations naturally when relevant
- Show genuine interest in the user's learning and project progress
- Build upon previous discussions to create continuity
- Adapt your technical depth to match the user's demonstrated skill level"""
        
        return prompt

# Create our AI agent
agent = MemoryEnhancedAgent(name="Bruno", personality="Brain_Powerhouse")
print("\n" + "="*60)
print("✅ AI Agent created successfully!")

 AI Agent 'Bruno' initialized!
 Personality: Brain_Powerhouse
 Connected to memory with 10 stored conversations

✅ AI Agent created successfully!


In [16]:

# we will Add these methods to our MemoryEnhancedAgent class
def add_memory_methods():
    """Add memory learning capabilities to the agent"""
    
    def search_memories(self, query, top_k=3):
        """Search for relevant memories based on the query"""
        print(f" {self.name} is searching memories for: '{query}'")
        
        results = self.collection.query(
            query_texts=[query],
            n_results=top_k
        )
        
        print(f" Found {len(results['documents'][0])} relevant memories")
        return results
    
    def save_conversation(self, user_message, ai_response, topic=None, user_level="intermediate"):
        """Save the conversation to memory for future reference"""
        try:
            # Generate unique ID for this conversation
            conversation_id = f"conv_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{str(uuid.uuid4())[:8]}"
            
            # Create conversation text
            conversation_text = f"User: {user_message}\n{self.name}: {ai_response}"
            
            # we are creating metadat for our ease
            metadata = {
                "topic": topic or "general",
                "user_level": user_level,
                "timestamp": datetime.now().isoformat(),
                "agent_name": self.name,
                "agent_personality": self.personality,
                "tech_stack": "conversation"
            }
            
            # Save to ChromaDB memory all in the db
            self.collection.add(
                documents=[conversation_text],
                metadatas=[metadata],
                ids=[conversation_id]
            )
            
            print(f" {self.name} saved conversation to memory!")
            return conversation_id
            
        except Exception as e:
            print(f" Error saving conversation: {e}")
            return None
    
    def format_memory_context(self, search_results):
        """Format search results into context for the LLM"""
        if not search_results['documents'][0]:
            return "No relevant conversation history found."
        
        context = "=== RELEVANT CONVERSATION HISTORY ===\n\n"
        
        for i, (doc, metadata) in enumerate(zip(
            search_results['documents'][0], 
            search_results['metadatas'][0]
        )):
            context += f"Memory {i+1}:\n{doc}\n"
            context += f"Topic: {metadata.get('topic', 'Unknown')}\n"
            context += f"User Level: {metadata.get('user_level', 'Unknown')}\n"
            context += f"Date: {metadata.get('timestamp', 'Unknown')}\n"
            context += "-" * 50 + "\n\n"
        
        return context
    
    # Add methods to the MemoryEnhancedAgent class
    MemoryEnhancedAgent.search_memories = search_memories
    MemoryEnhancedAgent.save_conversation = save_conversation
    MemoryEnhancedAgent.format_memory_context = format_memory_context
    
    print("✅ Memory learning methods added to Bruno!")

# Add the memory methods to our agent
add_memory_methods()

✅ Memory learning methods added to Bruno!


In [18]:
# below down will be our main agent fucntion for bruno

def create_agent_chat():
    """Create the main chat function for Bruno"""
    
    def agent_chat(self, user_message, save_to_memory=True, topic=None):
        """
        Main chat function with memory integration and learning
        """
        print("="*80)
        print(f" User: {user_message}")
        print("="*80)
        
        try:
            # Search for relevant memories
            memory_results = self.search_memories(user_message)
            memory_context = self.format_memory_context(memory_results)
            
            # Build enhanced prompt with personality + memory
            personality_prompt = self.get_personality_prompt()
            
            system_prompt = f"""{personality_prompt}

{memory_context}

ENHANCED INSTRUCTIONS:
- Use the conversation history to provide personalized responses
- Reference relevant past discussions naturally
- Maintain your personality consistently
- Build upon the user's learning journey
- Show genuine interest in their progress
- If the memory context isn't relevant, focus on being helpful with your established personality

Remember: You are {self.name} with a {self.personality} personality. Always maintain this character while being helpful and contextual."""

            # ollama responded :Generate response with Ollama
            response = ollama.chat(
                model=self.model,
                messages=[
                    {'role': 'system', 'content': system_prompt},
                    {'role': 'user', 'content': user_message}
                ]
            )
            
            ai_response = response['message']['content']
            print(f"{self.name}: {ai_response}")
            
            # Save conversation to memory (if enabled)
            if save_to_memory:
                conversation_id = self.save_conversation(
                    user_message=user_message,
                    ai_response=ai_response,
                    topic=topic
                )
                print(f" Conversation saved with ID: {conversation_id}")
            
            return ai_response
            
        except Exception as e:
            error_msg = f" Sorry, I encountered an error: {e}"
            print(error_msg)
            return error_msg
    
    # Add the chat method to our MemoryEnhancedAgent class
    MemoryEnhancedAgent.chat = agent_chat
    
    print(" Bruno's main chat function is ready!")
    print(" Bruno can now have memory-enhanced conversations!")

# Add the chat function to Bruno
create_agent_chat()

# Test Bruno's capabilities below snippet we will use for testing bruno
print("\n" + "="*60)
print(" Meet Bruno - Your Memory-Enhanced AI Agent!")
print(f" Current memory: {agent.collection.count()} conversations")
print(" Ready for intelligent conversations with memory!")

 Bruno's main chat function is ready!
 Bruno can now have memory-enhanced conversations!

 Meet Bruno - Your Memory-Enhanced AI Agent!
 Current memory: 10 conversations
 Ready for intelligent conversations with memory!


## Now its time to test our "BRUNO"


In [19]:
#  Asking Bruno about Python (should remember your history)
print("\n TEST 1: Asking Bruno about Python")
agent.chat("Hi Bruno! Can you help me with Python programming?", topic="programming")


 TEST 1: Asking Bruno about Python
 User: Hi Bruno! Can you help me with Python programming?
 Bruno is searching memories for: 'Hi Bruno! Can you help me with Python programming?'
 Found 3 relevant memories
Bruno: Hello there, my friend! *warm smile* I'd be delighted to help you with Python programming. What's on your mind? Are you working on a specific project or do you have some questions about the language itself?

By the way, I recall our previous conversation about Python being your go-to language for data science. How has your experience been so far? Have you been able to tackle any exciting projects or challenges?
 Bruno saved conversation to memory!
 Conversation saved with ID: conv_20250830_220951_641f7de8


"Hello there, my friend! *warm smile* I'd be delighted to help you with Python programming. What's on your mind? Are you working on a specific project or do you have some questions about the language itself?\n\nBy the way, I recall our previous conversation about Python being your go-to language for data science. How has your experience been so far? Have you been able to tackle any exciting projects or challenges?"

In [20]:
#  Asking about machine learning (should connect to your tech interests)
print("\n TEST 2: Asking Bruno about Machine Learning")
agent.chat("Bruno, I'm interested in starting a machine learning project. Any suggestions?", topic="machine_learning")


 TEST 2: Asking Bruno about Machine Learning
 User: Bruno, I'm interested in starting a machine learning project. Any suggestions?
 Bruno is searching memories for: 'Bruno, I'm interested in starting a machine learning project. Any suggestions?'
 Found 3 relevant memories
Bruno: Starting a new machine learning journey! Exciting times ahead!

I recall our conversation from August 30th about AI/ML - you were at an intermediate level back then. Since then, we've covered some fascinating topics together.

Considering your interest in long-term memory systems for AI agents (from that same conversation), I'd like to suggest a project that incorporates concepts related to self-supervised learning and transformer models. Have you explored the recent advancements in language modeling using transformers?

You could try building a simple chatbot or conversational interface using a library like Hugging Face's Transformers or PyTorch. This would allow you to experiment with transformer-based archi

"Starting a new machine learning journey! Exciting times ahead!\n\nI recall our conversation from August 30th about AI/ML - you were at an intermediate level back then. Since then, we've covered some fascinating topics together.\n\nConsidering your interest in long-term memory systems for AI agents (from that same conversation), I'd like to suggest a project that incorporates concepts related to self-supervised learning and transformer models. Have you explored the recent advancements in language modeling using transformers?\n\nYou could try building a simple chatbot or conversational interface using a library like Hugging Face's Transformers or PyTorch. This would allow you to experiment with transformer-based architectures and their applications, such as sequence-to-sequence models.\n\nAnother idea is to explore the world of recommender systems, which often rely on matrix factorization techniques and embedding methods. You could work on building a simple recommendation engine using l